# Retrieval Augmented Generation using Amazon DocumentDB

[Retrieval Augmented Generation](https://arxiv.org/abs/2005.11401) is a process that combines retrieval-based models and generative models to enhance natural language generation by retrieving relevant information and incorporating it into the generation process. 

In this lab we are going to be writing a RAG application code that allows user to ask questions about various wines so they can make a purchasing decision. We will use the semantic search (*vector search*) capability within DocumentDB to retrieve the best matching wine reviews and provide that to LLM for answering user's questions.

## 1. Lab Pre-requisites

#### a. Download and install python dependencies

For this notebook we require the use of a few libraries. We'll use the Python clients for DocumentDB and SageMaker, and Python frameworks for generating text embeddings.

In [ ]:
# Install required libraries
!pip install pymongo boto3 pandas --quiet
!pip install sagemaker --upgrade --quiet
!pip install requests_aws4auth --quiet
!pip install deprecated --quiet

#Progress bar for for loop
!pip install alive-progress  --quiet

# import PyTorch and confirm that the latest version of PyTorch is running. 
# The version should already be 1.13.1 or higher. If not, we will restart the kernel.

import torch
pytorch_version = torch.__version__
print( f"Pytorch version: {pytorch_version}")

def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
    
if pytorch_version.startswith('1.1'):
    from IPython.display import display_html


#### b. Import libraries & initialize resource information
The line below will import all the relevant libraries and modules used in this notebook.

In [ ]:
# Load the required libraries
import pymongo
import boto3
import os
import time
import json
import pandas as pd
from tqdm import tqdm
import sagemaker
import urllib.request

from sagemaker import get_execution_role
import random 
import string
import s3fs
from urllib.parse import urlparse
from IPython.display import display, HTML
from alive_progress import alive_bar
from requests_aws4auth import AWS4Auth
import requests 

#### c. Get CloudFormation stack output variables

We have preconfigured a few resources by creating a CloudFormation stack in the account. Names and ARN of these resources will be used within this lab. We are going to load some of the information variables here.


In [ ]:
# Create a Boto3 session
session = boto3.Session()

# Get the account id
account_id = boto3.client('sts').get_caller_identity().get('Account')

# Get the current region
region = session.region_name

cfn = boto3.client('cloudformation')

# Method to obtain output variables from Cloudformation stack. 
def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## Setup variables to use for the rest of the demo. Update the stack name with your cfn stack name
cloudformation_stack_name = "genai-data-foundation-workshop"

outputs = get_cfn_outputs(cloudformation_stack_name)
docDB_ep = outputs['DocumentDBClusterEndpoint']
s3_bucket = outputs['s3BucketTraining']
bedrock_inf_iam_role = outputs['BedrockBatchInferenceRole']
bedrock_inf_iam_role_arn = outputs['BedrockBatchInferenceRoleArn']
sagemaker_notebook_url = outputs['SageMakerNotebookURL']

# We will just print all the variables so you can easily copy if needed.
outputs

## 3. Prepare data
Below is the code that loads dataset of wine reviews, we'll use this data set to recommend wines that resemble the user provided description.

#### Sampling subset of the records to load into opensearch quickly
Since the data is composed of 129,000 records, it could take some time to convert them into vectors and load them in a vector store. Therefore, we will take a subset (300 records) of our data. We will add a variable called record_id which corresponds to the index of the record

In [ ]:
url = "https://raw.githubusercontent.com/davestroud/Wine/master/winemag-data-130k-v2.json"
df = pd.read_json(url)
df_sample = df.sample(300,random_state=37).reset_index()
df_sample['record_id'] = range(1, len(df_sample) + 1)
df_sample[:5]

## 3. Create a connection with Amazon DocumentDB cluster.
Next, we'll use PyMongo API to set up connection with Amazon DocumentDB cluster.

#### Retrieving credentials from Secrets manager
To avoid hard coding the user name and password in our code, we have dynamically generated a username and password at the time of deploying the cluster. This user name and password is stored in AWS Secrets Manager service. We will retrieve secret from Secrets Manager to establish DocumentDB connection. You also need to download the Amazon DocumentDB Certificate Authority (CA) certificate, which is required to authenticate your connection to the DocumentDB cluster.

In [ ]:
#Download the Amazon DocumentDB Certificate Authority (CA) certificate required to authenticate to your cluster 
urllib.request.urlretrieve("https://truststore.pki.rds.amazonaws.com/global/global-bundle.pem", "/home/ec2-user/global-bundle.pem")

In [ ]:
# Get credentials from AWS Secrets Manager
kms = boto3.client('secretsmanager')
docDB_credentials = json.loads(kms.get_secret_value(SecretId=outputs['DBSecret'])['SecretString'])

# Set up a connection to your Amazon DocumentDB cluster and creating the database.
client = pymongo.MongoClient(outputs['DocumentDBClusterEndpoint'],
                                port=27017,
                                username=docDB_credentials['username'],
                                password=docDB_credentials['password'],
                                retryWrites=False,
                                tls='true',
                                tlsCAFile="/home/ec2-user/global-bundle.pem") #Check the path as per your destination
db = client.semantic_rag_demo
collection = db.wine_knowledge_base
collection.drop()

## 4. Using Amazon Bedrock Titan Text embedding to convert text to vectors
Amazon Bedrock service offers Amazon Titan Text embedding v2 model that generates vector embeddings for text. This model will be used as our primary model for embeddings.

#### Helper method to invoke Titan Text embedding model in Amazon Bedrock
Creating a helper method in python to invoke Amazon Titan Text v2 embedding model. We will update `df_sample` data frame and add a new column called `description_vector` in it. Once this cell is executed, our data frame will be ready to load into Amazon DocumentDB. It may take a couple minutes to complete execution.

In [ ]:
import boto3
import pandas as pd
import os
from typing import Optional

# External Dependencies:
import boto3
from botocore.config import Config


bedrock_client = boto3.client(
    "bedrock-runtime", 
    region, 
    endpoint_url=f"https://bedrock-runtime.{region}.amazonaws.com"
)


def add_embeddings_to_df(df, text_column):

    # Create an empty list to store embeddings
    embeddings = []

    # Iterate over the text in the specified column
    for text in df[text_column]:
        embedding = embed_phrase(text)
        embeddings.append(embedding)
        

    # Add the embeddings as a new column to the DataFrame
    df['description_vector'] = embeddings

    return df

def embed_phrase( text ):
        
    model_id = "amazon.titan-embed-text-v2:0"  # 
    accept = "application/json"
    contentType = "application/json"

    # Prepare the request payload
    request_payload = json.dumps({"inputText": text})


    response = bedrock_client.invoke_model(body=request_payload, modelId=model_id, accept=accept, contentType=contentType)

    # Extract the embedding from the response
    response_body = json.loads(response.get('body').read())


    # Append the embedding to the list
    embedding = response_body.get("embedding")
    return embedding

df_sample = add_embeddings_to_df(df_sample, 'description')

df_sample[:5]

#### Let's try to create an embedding of a simple input text
You can see its an array of floating point numbers. While it does not make sense to human eye/brain, this array of numbers captures the semantics and knowledge of the text and that can be later used to compare two different text blocks. This method will be used to convert our query to a vector representation.

In [ ]:
## Create an vector embedding for input text
input_text = "A wine that pairs well with turkey breast?"

embedding = embed_phrase(input_text)

#printing text and embedding

print(f"{input_text=}")

#only printing first 10 dimensions of the 1024 dimension vector 
print(f"{embedding[:10]=}")

## 5. Load the raw data into the DocumentDB collection
Next, Let's load the wine review data and embeddings into the `wine_knowledge_base` collection. This step will create a collection (`wine_knowledge_base`) to store the wine data. The collection will include our embeddings in the description_vector field, which will be used later for semantic search, along with all other fields present in the dataset.

In [ ]:
collection.insert_many(df_sample.to_dict('records'))

To validate the load, we'll query the number of documents in the collection.The collection should contain 300 documents, or the number specified earlier during sampling.

In [ ]:
count = collection.count_documents({})
print("Records found: %d." % count)

## 6. Create a HNSW index for the Vector search   

Create a vector index using the `createIndex` command. DocumentDB performs an approximate nearest neighbor (ANN)search and supports both Hierarchical Navigable Small World (HNSW) indexing and Inverted File with Flat Compression (IVFFlat) indexing methods. The feature supports three distance metrics: Euclidean, cosine, and inner product (dotProduct). For this lab, we will create an HNSW index on the `description_vector` embedding field, using the cosine distance metric for similarity calculations.


In [ ]:
#Creating HNSW vector search index. You can set the parameters as per your performance and recall requirements.

collection.create_index ([("description_vector","vector")], 
    vectorOptions= {
        "type": "hnsw", 
        "similarity": "cosine",
        "dimensions": 1024,
        "m": 16,
        "efConstruction": 64},
    name="wine_hnsw_index")


Let's verify the created index information

In [ ]:
collection.index_information()

## 7. Search vector with "Semantic Search" 

Now we can define a helper function to execute the search query for us to find a wine whose review most closely matches the requested description. `retrieve_document_with_semantic_search` embeds the search phrase, searches the index for the closest matching vector, and returns the top result.


In [ ]:
def retrieve_document_with_semantic_search(phrase, n=3):
    search_vector = embed_phrase(phrase)
    
    #Setting up of projection parameters
    projection = {
        "_id":0,
        "description": 1, 
        "winery": 1,
        "points": 1, 
        "designation": 1,
        "country": 1, 
        "variety": 1, 
        "title": 1
    }
    
    docs = collection.aggregate([{
        '$search': {
            "vectorSearch" : {
                "vector" : search_vector,
                "path": "description_vector",
                "similarity": "cosine",
                "k": 2
            }
        }
    }, {"$project": projection} ])
    
    return list(docs)


Use the semantic search to get similar records with the sample question

In [ ]:
question_on_wine="Best Australian wine that goes great with steak?"

example_response = retrieve_document_with_semantic_search(question_on_wine)

print(example_response)


## 8. Prepare a method to call Amazon Bedrock - Anthropic Claude Sonnet model

Now we will define a function to call LLM to answer user's question. As LLM is trained with general purpose data, it may not have your wine review knowledge. While it may be able to answer, it may not be an answer that your business prefers. For example. in your case, you would not want it to recommend a wine that you do not stock. So the recommendation has to be one of the wines from your collection i.e. the 300 reviewed wines that we loaded. 

After defining this function we will call it to see how LLM answers questions without the wine review data.

In [ ]:
def query_llm_endpoint_with_json_payload(encoded_json):

    # Create a Bedrock Runtime client
    bedrock_client = boto3.client('bedrock-runtime')
    # Set the model ID for Claude 3 Sonnet
    model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
    accept = 'application/json'
    content_type = 'application/json'


    try:
        # Invoke the model with the native request payload
        response = bedrock_client.invoke_model(
            modelId=model_id,
            body=str.encode(str(encoded_json)),
            accept = accept,
            contentType=content_type
        )

        # Decode the response body
        response_body = json.loads(response.get('body').read())
        return response_body
    except Exception as e:
        print(f"Error: {e}")
        return none

def query_llm(system, user_question):
    # Define the prompt for the model
    prompt = "Write a sonnet about the beauty of nature."

    # Prepare the model's payload
    payload = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 10000,
        "system": system,
        "messages": [
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": f"{user_question}"
                }
              ]
            }
          ]
        })
    


    query_response = query_llm_endpoint_with_json_payload(payload)

    return query_response['content'][0]['text']


Let's check the generated result for a wine recommendation. It may not be one of the wine that we stock.

In [ ]:
def query_llm_without_rag(question):
    
    #Claude model has 2 parts of the prompt. System prompt guides the model what role to play
    system_prompt = f"You are a sommelier that uses their vast knowledge of wine to make great recommendations people will enjoy."
    
    #User prompt is the engineer prompt that has the context that model should reference to answer questions
    user_prompt = (
        f" As a sommelier, you must include the wine variety, the country of origin, "
        f"and a colorful description relating to the customer question."
        f"\n Customer question: {question}"
        f"\n Please provide name of the wine at the end of the answer, in a new line, in format Wine name: <wine name>"
    )
    return query_llm(system_prompt, user_prompt)


question_on_wine="Best Australian wine that goes great with steak?"

print(f"The recommened wine from LLM without RAG: \n\n{query_llm_without_rag(question_on_wine)}\n")

#### Testing for hallucination. 
Let's copy the wine name from the last line and paste it in the question variable below to see if we have this wine in our stock. Please review the list of wines that are returned. They may be from portugal but not exactly the one we have been recommended by the model. That's called hallucination, as model came up with an arbitrary recommendation with its general purpose knowledge.

__Note:__ If you do not see the wine name recommended by model above in the `wine_name` variable below, you should replace it so we can verify that the wine recommended is not in our HNSW index. 

In [ ]:
wine_name = "Penfolds Bin 389 Kalimna Shiraz"
example_request = retrieve_document_with_semantic_search(wine_name)
print("Matching wine records in our reviews:")
print(example_request)

## 9. Retrieval Augmented Generation
---
To resolve LLM hallunination problem, we can more context to LLM so that LLM can use context information to fine the model and generated factual result. RAG is one of the solution to the LLM hallucination. 


#### Create a prompt for the LLM using the search results from DocumentDB vector search (RAG)

We will be using the Anthropic Claude Sonnet 3 model with one-shot prompting technique. Within instructions to the model in the prompt, we will provide a sample wine review and how model should use to answer user's question. At the end of the prompt wine reviews retrieved from Amazon DocumentDB will be included for model to use. 

Before querying the model, the below function `generate_rag_based_system_prompt` is used to put together user prompt. The function takes an input string and performs a vector search on the `wine_knowledge_base` collection, specifically on the description_vector embedding field, to find a matching wine. It then composes the user prompt for the LLM.

System prompt defines the role that LLM plays.

User prompt contains the instructions and the context information that LLM model uses to answer user's question.

The prompt is in the following format:

**SYSTEM PROMPT:**

```
You are a sommelier that uses their vast knowledge of wine to make great recommendations people will enjoy. 
```


**USER PROMPT**
```
As a sommelier, you must include the wine variety, the country of origin, and a colorful description relating to the user's question.

Data:{'description': 'This perfumey white dances in intense and creamy layers of stone fruit and vanilla, remaining vibrant and balanced from start to finish. The generous fruit is grown in the relatively cooler Oak Knoll section of the Napa Valley. This should develop further over time and in the glass.', 'winery': 'Darioush', 'points': 92, 'designation': None, 'country': 'US'}

Recommendation:I have a wonderful wine for you. It's a dry, medium bodied white wine from Darioush winery in the Oak Knoll section of Napa Valley, US. It has flavors of vanilla and oak. It scored 92 points in wine spectator.

Data: {retrieved_documents}

Question from the user as is
```



### package the prompt and query the LLM
We will create a final function to query the LLM with the prompt. `query_llm_with_rag` is a function that calls LLM in a RAG.

`query_llm_with_rag` combines everything we've done in this module. It does all of the following:
- searches the DocumentDB HNSW index with semantic search for the relevant wine with `description_vector`
- generate an LLM prompt from the search results
- queriy the LLM with RAG for a response

In [ ]:
def query_llm_with_rag(user_question):
    retrieved_documents = retrieve_document_with_semantic_search(user_question)
    one_shot_description_example = "{'description': 'This perfumey white dances in intense and creamy layers of stone fruit and vanilla, remaining vibrant and balanced from start to finish. The generous fruit is grown in the relatively cooler Oak Knoll section of the Napa Valley. This should develop further over time and in the glass.', 'winery': 'Darioush', 'points': 92, 'designation': None, 'country': 'US'}"
    one_shot_response_example = "I have a wonderful wine for you. It's a dry, medium bodied white wine from Darioush winery in the Oak Knoll section of Napa Valley, US. It has flavors of vanilla and oak. It scored 92 points in wine spectator."
    system_prompt= "You are a sommelier that uses vast knowledge of wine to make great recommendations people will enjoy"
    user_prompt = (
        f"As a sommelier, you must include the wine variety, the country of origin, and a colorful description relating to the user question. You are must pick a wine in \"Wine data\" section only, one that matches best the customer question. Do not suggest anything outside of the wine data provided. You don't necessarily have to pick the top rated wine if its not best suited for customer question.\n"
        f"Wine data: {one_shot_description_example} \n Recommendation: {one_shot_response_example} \n"
        f"Wine data: {retrieved_documents} \n"
        f"Customer Question: {user_question} \n"        
    )
    response = query_llm(system_prompt, user_prompt)
    return response

#### And finally, let's call the function and get a wine recommendation.

In [ ]:
question_on_wine="Best Australian wine that goes great with steak?"
recommendation = query_llm_with_rag(question_on_wine)
print(recommendation)

print(f"\n\ndocuments retrieved for above recommendations were \n\n{json.dumps(retrieve_document_with_semantic_search(question_on_wine), indent=4)}")

#### Let's change it to Italian wine - it should produce a matching result.
We will call the same method again to see if there is an italian wine in our catalog.

In [ ]:
question_on_wine="Best Italian wine that goes great with steak?"
recommendation = query_llm_with_rag(question_on_wine)
print(recommendation)

print(f"\n\ndocuments retrieved for above recommendations were \n\n{json.dumps(retrieve_document_with_semantic_search(question_on_wine), indent=4)}")

You might notice that we asked for Australian wines that goes well with steak and we do not have any such wine in our collection. Therefore the model politely excuses. You may change the question and see how LLM recommends a wine from our select list that best suites your question.

### Conclusion
In this lab you built a simple wine recommendation chatbot. In this particular lab you used Amazon Bedrock titan v2 model to create vector embedding for our data. Then we loaded this data in to a DocumentDB collection with a HNSW index on `description_vector` field. At search time, we used Amazon Titan v2 model again to convert our query question into vector embedding and used semantic search to retrieve results. These results were then passed on to Anthropic Claude Sonnet 3 model which was able to recommend us a wine from within our catalog. 

## Lab finished - you may now go back to lab instructions section